<a href="https://colab.research.google.com/github/hargurjeet/MachineLearning/blob/master/ML_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install opendatasets

In [35]:
import opendatasets as od
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import os
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [36]:
od.download('https://www.kaggle.com/jsphyg/weather-dataset-rattle-package')

Skipping, found downloaded files in "./weather-dataset-rattle-package" (use force=True to force download)


In [37]:
raw_df = pd.read_csv('weather-dataset-rattle-package/weatherAUS.csv')
raw_df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,N,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [38]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [39]:
raw_df.dropna(subset=['RainTomorrow'], inplace=True)

limiting the dataset due to the limited computing power

In [40]:
raw_df = raw_df[:1000]

In [41]:
numeric_cols = raw_df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = raw_df.select_dtypes('object').columns.tolist()

In [42]:
print(numeric_cols)
print(categorical_cols)

['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']
['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']


Due to the computational limitation I am resticting the records to first 1000

In [43]:
print('Null values within the Dataset with numberical columns')
print(raw_df[numeric_cols].isna().sum())
print()
print('Null values within the Dataset with categroical columns')
print(raw_df[categorical_cols].isna().sum())

Null values within the Dataset with numberical columns
MinTemp             5
MaxTemp             1
Rainfall           10
Evaporation      1000
Sunshine         1000
WindGustSpeed       3
WindSpeed9am        3
WindSpeed3pm        3
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Cloud9am          609
Cloud3pm          611
Temp9am             0
Temp3pm             0
dtype: int64

Null values within the Dataset with categroical columns
Date              0
Location          0
WindGustDir       3
WindDir9am      197
WindDir3pm       22
RainToday        10
RainTomorrow      0
dtype: int64


Implementing imputer to fill emtpy values using mean for numberical columns

In [44]:
raw_df[numeric_cols]

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008,5.6,17.8,0.0,NaN,NaN,19.0,4.0,13.0,89.0,59.0,1020.5,1016.4,NaN,NaN,10.2,17.4
1009,10.2,16.0,0.0,NaN,NaN,13.0,9.0,4.0,87.0,80.0,1014.8,1012.6,NaN,8.0,13.0,15.0
1010,11.1,21.4,4.2,NaN,NaN,41.0,7.0,7.0,88.0,53.0,1013.2,1010.1,NaN,NaN,13.3,19.9
1011,8.7,21.8,0.0,NaN,NaN,33.0,4.0,24.0,79.0,50.0,1012.2,1009.5,NaN,NaN,13.5,21.3


In [46]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = 'constant').fit(raw_df[numeric_cols])

raw_df.loc[:, (numeric_cols)] = imputer.transform(raw_df[numeric_cols])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [47]:
raw_df[numeric_cols].isna().sum()

MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustSpeed    0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
dtype: int64

### Scaling Numeric Features

In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(raw_df[numeric_cols])
raw_df.loc[:, (numeric_cols)] = scaler.transform(raw_df[numeric_cols])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [55]:
raw_df[numeric_cols].describe().loc[['min', 'max']]

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Encoding Categorical Data

In [66]:
raw_df.RainTomorrow.value_counts()

No     772
Yes    228
Name: RainTomorrow, dtype: int64

In [ ]:
from sklearn.preprocessing import OneHotEncoder

raw_df[categorical_cols] = raw_df[categorical_cols].fillna('unknown')
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(raw_df[categorical_cols])

encoded_cols = list(encoder.get_feature_names(categorical_cols))

raw_df[encoded_cols] = encoder.transform(raw_df[categorical_cols])

In [59]:
raw_df[encoded_cols].columns

Index(['Date_2008-12-01', 'Date_2008-12-02', 'Date_2008-12-03',
       'Date_2008-12-04', 'Date_2008-12-05', 'Date_2008-12-06',
       'Date_2008-12-07', 'Date_2008-12-08', 'Date_2008-12-09',
       'Date_2008-12-10',
       ...
       'WindDir3pm_SW', 'WindDir3pm_W', 'WindDir3pm_WNW', 'WindDir3pm_WSW',
       'WindDir3pm_unknown', 'RainToday_No', 'RainToday_Yes',
       'RainToday_unknown', 'RainTomorrow_No', 'RainTomorrow_Yes'],
      dtype='object', length=1057)

In [68]:
raw_df[encoded_cols].head()

,Date_2008-12-01,Date_2008-12-02,Date_2008-12-03,Date_2008-12-04,Date_2008-12-05,Date_2008-12-06,Date_2008-12-07,Date_2008-12-08,Date_2008-12-09,Date_2008-12-10,Date_2008-12-11,Date_2008-12-12,Date_2008-12-13,Date_2008-12-14,Date_2008-12-16,Date_2008-12-17,Date_2008-12-18,Date_2008-12-19,Date_2008-12-20,Date_2008-12-21,Date_2008-12-22,Date_2008-12-23,Date_2008-12-24,Date_2008-12-25,Date_2008-12-26,Date_2008-12-27,Date_2008-12-28,Date_2008-12-29,Date_2008-12-30,Date_2008-12-31,Date_2009-01-01,Date_2009-01-02,Date_2009-01-03,Date_2009-01-04,Date_2009-01-05,Date_2009-01-06,Date_2009-01-07,Date_2009-01-08,Date_2009-01-09,Date_2009-01-10,Date_2009-01-11,Date_2009-01-12,Date_2009-01-13,Date_2009-01-14,Date_2009-01-15,Date_2009-01-16,Date_2009-01-17,Date_2009-01-18,Date_2009-01-19,Date_2009-01-20,Date_2009-01-21,Date_2009-01-22,Date_2009-01-23,Date_2009-01-24,Date_2009-01-25,Date_2009-01-26,Date_2009-01-27,Date_2009-01-28,Date_2009-01-29,Date_2009-01-30,Date_2009-01-31,Date_2009-02-01,Date_2009-02-02,Date_2009-02-03,Date_2009-02-04,Date_2009-02-05,Date_2009-02-06,Date_2009-02-07,Date_2009-02-08,Date_2009-02-09,Date_2009-02-10,Date_2009-02-11,Date_2009-02-12,Date_2009-02-13,Date_2009-02-14,Date_2009-02-15,Date_2009-02-16,Date_2009-02-17,Date_2009-02-18,Date_2009-02-19,Date_2009-02-20,Date_2009-02-21,Date_2009-02-22,Date_2009-02-23,Date_2009-02-24,Date_2009-02-25,Date_2009-02-26,Date_2009-02-27,Date_2009-02-28,Date_2009-03-01,Date_2009-03-02,Date_2009-03-03,Date_2009-03-04,Date_2009-03-05,Date_2009-03-06,Date_2009-03-07,Date_2009-03-08,Date_2009-03-09,Date_2009-03-10,Date_2009-03-11,Date_2009-03-12,Date_2009-03-13,Date_2009-03-14,Date_2009-03-15,Date_2009-03-16,Date_2009-03-17,Date_2009-03-18,Date_2009-03-19,Date_2009-03-20,Date_2009-03-21,Date_2009-03-22,Date_2009-03-23,Date_2009-03-24,Date_2009-03-25,Date_2009-03-26,Date_2009-03-27,Date_2009-03-28,Date_2009-03-29,Date_2009-03-30,Date_2009-03-31,Date_2009-04-01,Date_2009-04-02,Date_2009-04-03,Date_2009-04-04,Date_2009-04-05,Date_2009-04-06,Date_2009-04-07,Date_2009-04-08,Date_2009-04-09,Date_2009-04-10,Date_2009-04-11,Date_2009-04-12,Date_2009-04-13,Date_2009-04-14,Date_2009-04-15,Date_2009-04-16,Date_2009-04-17,Date_2009-04-18,Date_2009-04-19,Date_2009-04-20,Date_2009-04-21,Date_2009-04-22,Date_2009-04-23,Date_2009-04-24,Date_2009-04-25,Date_2009-04-26,Date_2009-04-27,Date_2009-04-28,Date_2009-04-29,Date_2009-04-30,Date_2009-05-01,Date_2009-05-02,Date_2009-05-03,Date_2009-05-04,Date_2009-05-05,Date_2009-05-06,Date_2009-05-07,Date_2009-05-08,Date_2009-05-09,Date_2009-05-10,Date_2009-05-11,Date_2009-05-12,Date_2009-05-13,Date_2009-05-14,Date_2009-05-15,Date_2009-05-16,Date_2009-05-17,Date_2009-05-18,Date_2009-05-19,Date_2009-05-20,Date_2009-05-21,Date_2009-05-22,Date_2009-05-23,Date_2009-05-24,Date_2009-05-25,Date_2009-05-26,Date_2009-05-27,Date_2009-05-28,Date_2009-05-29,Date_2009-05-30,Date_2009-05-31,Date_2009-06-01,Date_2009-06-02,Date_2009-06-03,Date_2009-06-04,Date_2009-06-05,Date_2009-06-06,Date_2009-06-07,Date_2009-06-08,Date_2009-06-09,Date_2009-06-10,Date_2009-06-11,Date_2009-06-12,Date_2009-06-13,Date_2009-06-14,Date_2009-06-15,Date_2009-06-16,Date_2009-06-17,Date_2009-06-18,Date_2009-06-19,Date_2009-06-20,Date_2009-06-21,Date_2009-06-22,Date_2009-06-23,Date_2009-06-24,Date_2009-06-25,Date_2009-06-26,Date_2009-06-27,Date_2009-06-28,Date_2009-06-29,Date_2009-06-30,Date_2009-07-01,Date_2009-07-02,Date_2009-07-03,Date_2009-07-04,Date_2009-07-05,Date_2009-07-06,Date_2009-07-07,Date_2009-07-08,Date_2009-07-09,Date_2009-07-10,Date_2009-07-11,Date_2009-07-12,Date_2009-07-13,Date_2009-07-14,Date_2009-07-15,Date_2009-07-16,Date_2009-07-17,Date_2009-07-18,Date_2009-07-19,Date_2009-07-20,Date_2009-07-21,Date_2009-07-22,Date_2009-07-23,Date_2009-07-24,Date_2009-07-25,Date_2009-07-26,Date_2009-07-27,Date_2009-07-28,Date_2009-07-29,Date_2009-07-30,Date_2009-07-31,Date_2009-08-01,Date_2009-08-02,Date_2009-08-03,Date_2009-08-04,Date_2009-08-05,Date_2009-08-06,Date_2009-08-07,Date_2009-08-08

In [69]:
raw_df = raw_df[numeric_cols + encoded_cols]

raw_df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Date_2008-12-01,Date_2008-12-02,Date_2008-12-03,Date_2008-12-04,Date_2008-12-05,Date_2008-12-06,Date_2008-12-07,Date_2008-12-08,Date_2008-12-09,Date_2008-12-10,Date_2008-12-11,Date_2008-12-12,Date_2008-12-13,Date_2008-12-14,Date_2008-12-16,Date_2008-12-17,Date_2008-12-18,Date_2008-12-19,Date_2008-12-20,Date_2008-12-21,Date_2008-12-22,Date_2008-12-23,Date_2008-12-24,Date_2008-12-25,Date_2008-12-26,Date_2008-12-27,Date_2008-12-28,Date_2008-12-29,Date_2008-12-30,Date_2008-12-31,Date_2009-01-01,Date_2009-01-02,Date_2009-01-03,Date_2009-01-04,Date_2009-01-05,Date_2009-01-06,Date_2009-01-07,Date_2009-01-08,Date_2009-01-09,Date_2009-01-10,Date_2009-01-11,Date_2009-01-12,Date_2009-01-13,Date_2009-01-14,Date_2009-01-15,Date_2009-01-16,Date_2009-01-17,Date_2009-01-18,Date_2009-01-19,Date_2009-01-20,Date_2009-01-21,Date_2009-01-22,Date_2009-01-23,Date_2009-01-24,Date_2009-01-25,Date_2009-01-26,Date_2009-01-27,Date_2009-01-28,Date_2009-01-29,Date_2009-01-30,Date_2009-01-31,Date_2009-02-01,Date_2009-02-02,Date_2009-02-03,Date_2009-02-04,Date_2009-02-05,Date_2009-02-06,Date_2009-02-07,Date_2009-02-08,Date_2009-02-09,Date_2009-02-10,Date_2009-02-11,Date_2009-02-12,Date_2009-02-13,Date_2009-02-14,Date_2009-02-15,Date_2009-02-16,Date_2009-02-17,Date_2009-02-18,Date_2009-02-19,Date_2009-02-20,Date_2009-02-21,Date_2009-02-22,Date_2009-02-23,Date_2009-02-24,Date_2009-02-25,Date_2009-02-26,Date_2009-02-27,Date_2009-02-28,Date_2009-03-01,Date_2009-03-02,Date_2009-03-03,Date_2009-03-04,Date_2009-03-05,Date_2009-03-06,Date_2009-03-07,Date_2009-03-08,Date_2009-03-09,Date_2009-03-10,Date_2009-03-11,Date_2009-03-12,Date_2009-03-13,Date_2009-03-14,Date_2009-03-15,Date_2009-03-16,Date_2009-03-17,Date_2009-03-18,Date_2009-03-19,Date_2009-03-20,Date_2009-03-21,Date_2009-03-22,Date_2009-03-23,Date_2009-03-24,Date_2009-03-25,Date_2009-03-26,Date_2009-03-27,Date_2009-03-28,Date_2009-03-29,Date_2009-03-30,Date_2009-03-31,Date_2009-04-01,Date_2009-04-02,Date_2009-04-03,Date_2009-04-04,Date_2009-04-05,Date_2009-04-06,Date_2009-04-07,Date_2009-04-08,Date_2009-04-09,Date_2009-04-10,Date_2009-04-11,Date_2009-04-12,Date_2009-04-13,Date_2009-04-14,Date_2009-04-15,Date_2009-04-16,Date_2009-04-17,Date_2009-04-18,Date_2009-04-19,Date_2009-04-20,Date_2009-04-21,Date_2009-04-22,Date_2009-04-23,Date_2009-04-24,Date_2009-04-25,Date_2009-04-26,Date_2009-04-27,Date_2009-04-28,Date_2009-04-29,Date_2009-04-30,Date_2009-05-01,Date_2009-05-02,Date_2009-05-03,Date_2009-05-04,Date_2009-05-05,Date_2009-05-06,Date_2009-05-07,Date_2009-05-08,Date_2009-05-09,Date_2009-05-10,Date_2009-05-11,Date_2009-05-12,Date_2009-05-13,Date_2009-05-14,Date_2009-05-15,Date_2009-05-16,Date_2009-05-17,Date_2009-05-18,Date_2009-05-19,Date_2009-05-20,Date_2009-05-21,Date_2009-05-22,Date_2009-05-23,Date_2009-05-24,Date_2009-05-25,Date_2009-05-26,Date_2009-05-27,Date_2009-05-28,Date_2009-05-29,Date_2009-05-30,Date_2009-05-31,Date_2009-06-01,Date_2009-06-02,Date_2009-06-03,Date_2009-06-04,Date_2009-06-05,Date_2009-06-06,Date_2009-06-07,Date_2009-06-08,Date_2009-06-09,Date_2009-06-10,Date_2009-06-11,Date_2009-06-12,Date_2009-06-13,Date_2009-06-14,Date_2009-06-15,Date_2009-06-16,Date_2009-06-17,Date_2009-06-18,Date_2009-06-19,Date_2009-06-20,Date_2009-06-21,Date_2009-06-22,Date_2009-06-23,Date_2009-06-24,Date_2009-06-25,Date_2009-06-26,Date_2009-06-27,Date_2009-06-28,Date_2009-06-29,Date_2009-06-30,Date_2009-07-01,Date_2009-07-02,Date_2009-07-03,Date_2009-07-04,Date_2009-07-05,Date_2009-07-06,Date_2009-07-07,Date_2009-07-08,Date_2009-07-09,Date_2009-07-10,Date_2009-07-11,Date_2009-07-12,Date_2009-07-13,Date_2009-07-14,Date_2009-07-15,Date_2009-07-16,Date_2009-07-17,Date_2009-07-18,Date_2009-07-19,Date_2009-07-20,Date_2009-07-21,Date_2009-07-22,Date_2009-07-23,Date_2009-07-24,Date_2009-07-25,Date_2009-07-26,Date_2009-07-27,Date_2009-07-28,Date_20